In [ ]:
# Benchmarking Tool Interface
%load_ext autoreload
%autoreload 2



import ipywidgets as widgets
import json
from IPython.display import display, Markdown, clear_output
import matplotlib.pyplot as plt
import os



def load_model_config(filename="configs/models.json"):
    with open(filename, "r") as f:
        model_config = json.load(f)

    # catalogue = {"all": []}
    catalogue = {}
    for model in model_config["models"]:
        if "layers" in model:
            layers = model["layers"]
            for layer in layers:
                if layer not in catalogue:
                    catalogue[layer] = []
                catalogue[layer].append(model["name"])
    return catalogue

model_widgets = {}

def create_model_selection_widgets(model_catalogue):
    selected_models = []

    def on_value_change(change):
        selected_models.clear()
        for layer in model_widgets:
            selected_models.extend(model_widgets[layer].children[1].value)
        selected_models[:] = list(set(selected_models))  # Ensure unique selection

    def select_all_models(layer):
        model_widgets[layer].children[1].value = model_catalogue[layer]

    num_columns = 3  # Number of columns in the grid
    num_rows = (
        len(model_catalogue) + num_columns - 1
    ) // num_columns  # Calculate the number of rows needed

    grid = widgets.GridspecLayout(
        num_rows, num_columns, width="70%", grid_gap="50px"
    )  # Add grid_gap to reduce distance between columns
    row = 0
    col = 0
    for layer in model_catalogue:
        max_model_name_length = max(
            len(model) for models in model_catalogue.values() for model in models
        )
        model_widgets[layer] = widgets.VBox(
            [
                widgets.Label(
                    value=layer, layout=widgets.Layout(margin="0 0 1px 100px")
                ),
                widgets.SelectMultiple(
                    options=model_catalogue[layer],
                    value=[],
                    description="",
                    disabled=False,
                    layout=widgets.Layout(
                        width=f"{max_model_name_length * 9}px", margin="0 0 1px 75px"
                    ),
                ),
                widgets.Button(
                    description=f"All {layer}",
                    layout=widgets.Layout(margin="0 0 1px 75px"),
                    style=widgets.ButtonStyle(),
                ),
            ]
        )
        model_widgets[layer].children[1].observe(on_value_change, names="value")
        model_widgets[layer].children[2].on_click(
            lambda b, l=layer: select_all_models(l)
        )
        grid[row, col] = model_widgets[layer]
        col += 1
        if col == num_columns:
            col = 0
            row += 1

    # display(grid)
    return selected_models , grid

model_catalogue = load_model_config()


def load_hardware_configs(base_dir="../../hardware_automation/configs"):
    hardware_config = {}

    for root, dirs, files in os.walk(base_dir):
        for file in files:
            if file.endswith(".json"):
                file_path = os.path.join(root, file)
                with open(file_path, "r") as f:
                    config = json.load(f)
                    folder_name = os.path.basename(root)
                    if folder_name not in hardware_config:
                        hardware_config[folder_name] = []
                    hardware_config[folder_name].append(
                        {"hardware": file.split(".")[0], "config": config}
                    )

    return hardware_config



def configure_benchmark():
    # find the boards we are using through the selected hardwares
    boards = set()
    for hw in selected_hardware:
        for folder, hw_list in hardware_config.items():
            for hw_item in hw_list:
                if hw_item["hardware"] == hw:
                    board = hw_item["config"].get("board", "Z1")
                    boards.add(board)
                    break
    return list(boards)

selected_boards = []
def update_selected_boards():
    global selected_boards
    selected_boards = configure_benchmark()

    # Create a mapping from board to hardware
    global board_to_hardware_mapping
    board_to_hardware_mapping = {}
    for hw in selected_hardware:
        for folder, hw_list in hardware_config.items():
            for hw_item in hw_list:
                if hw_item["hardware"] == hw:
                    board = hw_item["config"].get("board", "Z1")
                    if board not in board_to_hardware_mapping:
                        board_to_hardware_mapping[board] = []
                    board_to_hardware_mapping[board].append(hw)
                    break

hardware_widgets = {}
def create_hardware_selection_widgets(hardware_config):

    selected_hardware = []

    def on_value_change(change):
        selected_hardware.clear()
        for folder in hardware_widgets:
            selected_hardware.extend(hardware_widgets[folder].children[1].value)
        selected_hardware[:] = list(set(selected_hardware))  # Ensure unique selection
        update_selected_boards()  # Update selected_boards when selected_hardware is updated

    def select_all_hardware(folder):
        hardware_widgets[folder].children[1].value = [
            hw["hardware"] for hw in hardware_config[folder]
        ]

    num_columns = 3  # Number of columns in the grid
    num_rows = (
        len(hardware_config) + num_columns - 1
    ) // num_columns  # Calculate the number of rows needed

    max_hardware_name_length = max(
        len(hw["hardware"])
        for folder in hardware_config
        for hw in hardware_config[folder]
    )
    grid = widgets.GridspecLayout(
        num_rows, num_columns, width="70%", grid_gap="50px"
    )  # Add grid_gap to reduce distance between columns
    row = 0
    col = 0
    for folder in hardware_config:
        hardware_widgets[folder] = widgets.VBox(
            [
                widgets.Label(
                    value=folder, layout=widgets.Layout(margin="0 0 1px 100px")
                ),
                widgets.SelectMultiple(
                    options=[hw["hardware"] for hw in hardware_config[folder]],
                    value=[],
                    description="",
                    disabled=False,
                    layout=widgets.Layout(
                        width=f"{max_hardware_name_length * 9}px", margin="0 0 1px 75px"
                    ),
                ),
                widgets.Button(
                    description=f"All {folder}",
                    layout=widgets.Layout(margin="0 0 1px 75px"),
                    style=widgets.ButtonStyle(),
                ),
            ]
        )
        hardware_widgets[folder].children[1].observe(on_value_change, names="value")
        hardware_widgets[folder].children[2].on_click(
            lambda b, f=folder: select_all_hardware(f)
        )
        grid[row, col] = hardware_widgets[folder]
        col += 1
        if col == num_columns:
            col = 0
            row += 1
    # display(grid)
    return selected_hardware , grid

hardware_config = load_hardware_configs()

def load_widget_values():
  try:
    with open("configs/widget_values.json", "r") as f:
      widget_values = json.load(f)
      init_widget.value = widget_values.get("init", False)
      skip_bench_widget.value = widget_values.get("skip_bench", False)
      bin_gen_widget.value = widget_values.get("bin_gen", False)
      skip_inf_diff_widget.value = widget_values.get("skip_inf_diff", False)
      collect_power_widget.value = widget_values.get("collect_power", False)
      test_run_widget.value = widget_values.get("test_run", False)
      name_widget.value = widget_values.get("name", "")
      num_runs_widget.value = widget_values.get("num_runs", 1)
      threads_widget.value = widget_values.get("threads", "1")
      sim_mode_widget.value = widget_values.get("sim_mode", False)
      for layer in model_widgets:
        model_widgets[layer].children[1].value = widget_values.get("model_widgets", {}).get(layer, [])
      for folder in hardware_widgets:
        hardware_widgets[folder].children[1].value = widget_values.get("hardware_widgets", {}).get(folder, [])
  except FileNotFoundError:
    print("No saved configuration found.")


def save_widget_values():
  widget_values = {
    "init": init_widget.value,
    "skip_bench": skip_bench_widget.value,
    "bin_gen": bin_gen_widget.value,
    "skip_inf_diff": skip_inf_diff_widget.value,
    "collect_power": collect_power_widget.value,
    "test_run": test_run_widget.value,
    "name": name_widget.value,
    "num_runs": num_runs_widget.value,
    "threads": threads_widget.value,
    "sim_mode": sim_mode_widget.value,
    "model_widgets": {layer: model_widgets[layer].children[1].value for layer in model_widgets},
    "hardware_widgets": {folder: hardware_widgets[folder].children[1].value for folder in hardware_widgets},
  }
  with open("configs/widget_values.json", "w") as f:
    json.dump(widget_values, f)


# SECDA Benchmarking Suite
title = widgets.HTML(value="<h2 style='margin-bottom: 0px;'>SECDA Benchmarking Suite</h2>")

# Model Selection Interface
model_title = widgets.HTML(value="<h3 style='margin-bottom: 0px;'>Model Selection Interface</h3>")

# Accelerator Selection Interface
hardware_title = widgets.HTML(value="<h3 style='margin-bottom: 0px;'>Accelerator Selection Interface</h3>")

# Benchmarking Tool Interface
benchmark_title = widgets.HTML(value="<h3 style='margin-bottom: 0px;'>Benchmarking Tool Interface</h3>")

selected_models, models_grid = create_model_selection_widgets(model_catalogue)
selected_hardware, hw_grid = create_hardware_selection_widgets(hardware_config)

# Configuration widgets
init_widget = widgets.Checkbox(value=False, description='Initialize the board', layout=widgets.Layout(margin="0 0 0 20px"), style=dict(description_width='initial'))
skip_bench_widget = widgets.Checkbox(value=False, description='Skip running experiment', layout=widgets.Layout(margin="0 0 0 20px"), style=dict(description_width='initial'))
bin_gen_widget = widgets.Checkbox(value=False, description='Generate binaries', layout=widgets.Layout(margin="0 0 0 20px"), style=dict(description_width='initial'))
skip_inf_diff_widget = widgets.Checkbox(value=False, description='Skip inference difference checks', layout=widgets.Layout(margin="0 0 0 20px"), style=dict(description_width='initial'))
collect_power_widget = widgets.Checkbox(value=False, description='Power collection', layout=widgets.Layout(margin="0 0 0 20px"), style=dict(description_width='initial'))
test_run_widget = widgets.Checkbox(value=False, description='Test run', layout=widgets.Layout(margin="0 0 0 20px"), style=dict(description_width='initial'))
name_widget = widgets.Text(value='', description='Name of the experiment', layout=widgets.Layout(margin="0 0 0 20px"), style=dict(description_width='initial'))
num_runs_widget = widgets.IntText(value=1, description='Number of runs', layout=widgets.Layout(margin="0 0 0 20px"), style=dict(description_width='initial'))
threads_widget = widgets.Text(value='1', description='Threads (comma-separated)', layout=widgets.Layout(margin="0 0 0 20px"), style=dict(description_width='initial'))
sim_mode_widget = widgets.Checkbox(value=False, description='Simulation Mode', layout=widgets.Layout(margin="0 0 0 20px"), style=dict(description_width='initial'))

# Create a horizontal box layout for the new widgets
exp_widgets_0 = widgets.HBox([name_widget, num_runs_widget, threads_widget])
exp_widgets_1 = widgets.HBox([init_widget, skip_bench_widget, bin_gen_widget,sim_mode_widget])
exp_widgets_2 = widgets.HBox([skip_inf_diff_widget, collect_power_widget, test_run_widget])

load_button = widgets.Button(description="Load Configuration")
load_button.on_click(lambda b: load_widget_values())

save_button = widgets.Button(description="Save Configuration")
save_button.on_click(lambda b: save_widget_values())

# Display the all widgets
exp_widgets = widgets.VBox([title, model_title, models_grid, hardware_title, hw_grid,
  benchmark_title,exp_widgets_0, exp_widgets_1, exp_widgets_2,
  widgets.HBox([load_button, save_button])
  ])


display(exp_widgets)

# =============================================================================
# Run the SECDA Benchmark Suite
# =============================================================================
from scripts.configure_utils import *
board_results = {}

out = widgets.Output(layout={"border": "1px solid black"})
def on_button_click(b):
    out.clear_output()
    clear_output()
    display(exp_widgets)
    display(start_terminal)
    board_results.clear()
    with out:
      sc = load_config("../../config.json")
      selected_threads = [
          int(thread.strip())
          for thread in threads_widget.value.split(",")
          if thread.strip()
      ]
      selected_num_runs = num_runs_widget.value
      exp_config = {
          "init": init_widget.value,
          "skip_bench": skip_bench_widget.value,
          "bin_gen": bin_gen_widget.value,
          "skip_inf_diff": skip_inf_diff_widget.value,
          "collect_power": collect_power_widget.value,
          "sim_mode": sim_mode_widget.value,
          "test_run": test_run_widget.value,
          "name": name_widget.value,
      }
      ret = run_benchmarking_suite(
          sc,
          selected_boards,
          selected_models,
          selected_hardware,
          selected_threads,
          selected_num_runs,
          hardware_config,
          exp_config,
          board_results,
          out,
      )

    for board in board_results:
      display(Markdown(f"### {board} Results"))
      display(board_results[board])

    # Assuming board_results contains the necessary data
    for board, result in board_results.items():
      if 'acc_layer' in result.columns and 'cpu_layers' in result.columns:
        plt.figure(figsize=(12, 8))
        plt.bar(result['runname'], result['acc_layer'], label='Accelerator Layer')
        plt.bar(result['runname'], result['cpu_layers'], bottom=result['acc_layer'], label='CPU Layer')
        plt.title(f'{board} Layer Performance')
        plt.xlabel('Run Name')
        plt.ylabel('Performance')
        plt.xticks(rotation=45, ha='right')
        plt.legend()
        plt.tight_layout()
        plt.show()
    

# Create the button
with out:
    start_button = widgets.Button(description="Start Benchmarking")
    start_button.on_click(on_button_click)

start_terminal = widgets.VBox([start_button,out])
display(start_terminal)

